In [209]:
import numpy as np
import pandas as pd
from scipy import fftpack
from scipy import signal
import datetime
import pickle
import time
%matplotlib inline

In [182]:
p_path="/home/takeyama/pywork/ipython/2016-05-23/"

In [11]:
cd ~/Documents/SyncRecord/cleaning-addingLABEL/

/home/takeyama/Documents/SyncRecord/cleaning-addingLABEL


In [18]:
class SensorData:
    
    def __init__(self):
        print "__class__"
        # raw data
        self._AccX=np.array([])
        self._AccY=np.array([])
        self._AccZ=np.array([])
        self._GyrX=np.array([])
        self._GyrY=np.array([])
        self._GyrZ=np.array([])
        self._Columns=np.array([])
        
        # fft data
        self._fft_AccX=np.array([])
        self._fft_AccY=np.array([])
        self._fft_AccZ=np.array([])
        self._fft_GyrX=np.array([])
        self._fft_GyrY=np.array([])
        self._fft_GyrZ=np.array([])

        # power spectol data
        self._power_AccX=np.array([])
        self._power_AccY=np.array([])
        self._power_AccZ=np.array([])
        self._power_GyrX=np.array([])
        self._power_GyrY=np.array([])
        self._power_GyrZ=np.array([])
        
        # flag exsist data
        self.Flag_exist_data=False
        
    def ImportCSV(self,csv_file):
        self.Flag_exist_data=True
        # design dataframe
        self._data = pd.read_csv(csv_file,encoding="SHIFT-JIS")
        self._data.columns=[u'Type',u'Time',u'AccX',u'AccY',u'AccZ',u'GyrX',u'GyrY',u'GyrZ']
        self._data.Time=pd.to_datetime(self._data.Time)
        self._data = pd.pivot_table(self._data,values=[u'AccX',u'AccY',u'AccZ',u'GyrX',u'GyrY',u'GyrZ'],index=[u'Time'])
        
        # convert numpy.darray
        self._AccX=self._data.AccX.values
        self._AccY=self._data.AccY.values
        self._AccZ=self._data.AccZ.values
        self._GyrX=self._data.GyrX.values
        self._GyrY=self._data.GyrY.values
        self._GyrZ=self._data.GyrZ.values
        self._Time=self._data.index.to_pydatetime().astype('datetime64[ns]')
        self._Columns=['Time','AccX','AccY','AccZ','GyrX','GyrY','GyrZ']
        self._fft_col=['fft_AccX','fft_AccY','fft_AccZ','fft_GyrX','fft_GyrY','fft_GyrZ']
        self._power_col=['power_AccX','power_AccY','power_AccZ','power_GyrX','power_GyrY','power_GyrZ']
    
    def ShowFlagExistData(self):
        return self.Flag_exist_data
    
    def GetColumns(self):
        return self._Columns

    def ShowAllDf(self):
        print self._AccX;print self._AccY;print self._AccZ;print self._GyrX;print self._GyrY;print self._GyrZ;print self._Time
        
    def _String2Var(self,Sname):
        if(Sname=='AccX'):return self._AccX
        elif(Sname=='AccY'):return self._AccY
        elif(Sname=='AccZ'):return self._AccZ
        elif(Sname=='GyrX'):return self._GyrX
        elif(Sname=='GyrY'):return self._GyrY
        elif(Sname=='GyrZ'):return self._GyrZ
        elif(Sname=='Time'):return self._Time
        else:print 'Wrong Axis Name'
    
    def _Time2Num(self,time):
        index=np.where(test._Time==np.datetime64(time) )
        return index[0][0]    
    
    def _Query(self,Sname,rng=[]):
        data = self._String2Var(Sname)
        return data[rng[0]:rng[1]]
            
    def GetLength(self,Sname):
        data = _String2Var(Sname)
        print Sname+' length ='+str( len(data) )
            
    def ShowQuery(self,Sname,rng=[]):
        data = self._String2Var(Sname)
        print Sname+':'+str( data[rng[0]:rng[1]])
            
    def _sliding_window(self,Sname,samp, overlap):
        count =0
        s =self._Time[0]
        start=self._Time2Num(s)
        g = s+np.timedelta64(samp*10,'ms')
        goal= self._Time2Num(g)
        yield self._Query(Sname,[start,goal])
        
        add=overlap*0.01
        
        while True:
            try:
                count +=1
                s =s+np.timedelta64(samp*10,'ms')
                start=self._Time2Num(s)
                g  =s+np.timedelta64(samp*10,'ms')
                goal= self._Time2Num(g)
                yield self._Query(Sname,[start,goal])
            except StopIteration:
                print '_sliding_window StopIteration'
                break   
            except IndexError:
                print '_sliding_window IndexError'
                break  
                
# Fast Frier transaction
    def _set_fft(self,Sname,array):
        #print array
        if(Sname=='fft_AccX'):   self._fft_AccX=np.append(self._fft_AccX,np.array([array]) )
        elif(Sname=='fft_AccY'): self._fft_AccY=np.append(self._fft_AccY,np.array([array]) )
        elif(Sname=='fft_AccZ'): self._fft_AccZ=np.append(self._fft_AccZ,np.array([array]) )
        elif(Sname=='fft_GyrX'): self._fft_GyrX=np.append(self._fft_GyrX,np.array([array]) )
        elif(Sname=='fft_GyrY'): self._fft_GyrY=np.append(self._fft_GyrY,np.array([array]) )
        elif(Sname=='fft_GyrZ'): self._fft_GyrZ=np.append(self._fft_GyrZ,np.array([array]) )
        else:print 'Wrong Axis Name'
            
    def _GetFFT(self,Sfft,samp):
        return np.load(p_path+'fft/'+Sfft+'_'+str(samp)+'.npz')['arr_0']
    
    def CalcFFT(self,samp,overlap=0.5):
        for n,f in zip( self._Columns[1:],self._fft_col):
            print 'start'+n+'->'+f
            sw = self._sliding_window(n,samp,overlap)
            count=0
            while True:
                try:
                    #print 'count ='+str(count)
                    d=sw.next()
                    # fftの直流成分を除くsample/2のみ
                    self._set_fft(f,fftpack.fft(d)[1:(samp/2)+1])     
                    count = count+1
                except StopIteration:
                    print 'CalcFFTStopIteration'
                    break   
                    
        self._fft_AccX=self._fft_AccX.reshape(len(self._fft_AccX)/(samp/2),(samp/2) )
        self._fft_AccY=self._fft_AccY.reshape(len(self._fft_AccY)/(samp/2),(samp/2) )
        self._fft_AccZ=self._fft_AccZ.reshape(len(self._fft_AccZ)/(samp/2),(samp/2) )
        self._fft_GyrX=self._fft_GyrX.reshape(len(self._fft_GyrX)/(samp/2),(samp/2) )
        self._fft_GyrY=self._fft_GyrY.reshape(len(self._fft_GyrY)/(samp/2),(samp/2) )
        self._fft_GyrZ=self._fft_GyrZ.reshape(len(self._fft_GyrZ)/(samp/2),(samp/2) )
        
        np.savez(p_path+'fft/'+'fft_AccX_'+str(samp),self._fft_AccX)
        np.savez(p_path+'fft/'+'fft_AccY_'+str(samp),self._fft_AccY)
        np.savez(p_path+'fft/'+'fft_AccZ_'+str(samp),self._fft_AccZ)
        np.savez(p_path+'fft/'+'fft_GyrX_'+str(samp),self._fft_GyrX)
        np.savez(p_path+'fft/'+'fft_GyrY_'+str(samp),self._fft_GyrY)
        np.savez(p_path+'fft/'+'fft_GyrZ_'+str(samp),self._fft_GyrZ)

# Spectol Power 
    def _GetPower(self,Spower,samp):
        return np.load(p_path+'power/'+Spower+'_'+str(samp)+'.npz')['arr_0']
    
    def _power(self,fft_array):
        p=lambda x,y : np.sqrt(x**2+y**2)
        power_array=np.array([])
        for vector in fft_array:
            tmp = p( np.real(vector),np.imag(vector) )
            tmp = tmp/np.sum(tmp)
            power_array = np.append(power_array,tmp)
        return power_array
        
    def CalcPower(self,samp,overlap=0.5):
        local_fft_AccX=np.load(p_path+'fft/'+'fft_AccX_'+str(samp)+'.npz')['arr_0']
        local_fft_AccY=np.load(p_path+'fft/'+'fft_AccY_'+str(samp)+'.npz')['arr_0']
        local_fft_AccZ=np.load(p_path+'fft/'+'fft_AccZ_'+str(samp)+'.npz')['arr_0']
        local_fft_GyrX=np.load(p_path+'fft/'+'fft_GyrX_'+str(samp)+'.npz')['arr_0']
        local_fft_GyrY=np.load(p_path+'fft/'+'fft_GyrY_'+str(samp)+'.npz')['arr_0']
        local_fft_GyrZ=np.load(p_path+'fft/'+'fft_GyrZ_'+str(samp)+'.npz')['arr_0']
           
        self._power_AccX = self._power(local_fft_AccX)
        self._power_AccY = self._power(local_fft_AccY)
        self._power_AccZ = self._power(local_fft_AccZ)
        self._power_GyrX = self._power(local_fft_GyrX)
        self._power_GyrY = self._power(local_fft_GyrY)
        self._power_GyrZ = self._power(local_fft_GyrZ)

        np.savez(p_path+'power/'+'power_AccX_'+str(samp),self._power_AccX)
        np.savez(p_path+'power/'+'power_AccY_'+str(samp),self._power_AccY)
        np.savez(p_path+'power/'+'power_AccZ_'+str(samp),self._power_AccZ)
        np.savez(p_path+'power/'+'power_GyrX_'+str(samp),self._power_GyrX)
        np.savez(p_path+'power/'+'power_GyrY_'+str(samp),self._power_GyrY)
        np.savez(p_path+'power/'+'power_GyrZ_'+str(samp),self._power_GyrZ)
        
# Kallback Librar Divergence
    def _KLD(self,vect1,vect2):
        f =lambda p,q : np.sum(p * np.log(p / q), axis=(p.ndim - 1))
        kld_array=np.array([])
        for vector1 in vect1:
            for vector2 in vect2:
                tmp = f( vector1, vector2 )
                kld_array = np.append(kld_array,tmp)
        return kld_array
    
    def CalcKLD(self,Sarray1,Sarray2,samp):
        dist1 = self._GetPower(Sarray1,samp)
        dist2 = self._GetPower(Sarray2,samp)
        
        self._kld = self._KLD(dist1,dist2)    
        np.savez(p_path+'kld/'+Sarray1+'_VS_'+Sarray2,self._kld)
        
    def ShowKLD(self,Sarray1,Sarray2,samp):
        tmp = np.load(p_path+'kld/'+Sarray1+'_VS_'+Sarray2+'.npz')['arr_0']
        print Sarray1+'_VS_'+Sarray2+'Kullback–Leibler Divergence'
        print 'shape ='+str(tmp.shape)
        print tmp
        
    def _JSD(self,vect1,vect2):
        f =lambda p,q : 0.5*self._KLD(p,(0.5*(p+q)) )+0.5*self._KLD(q,(0.5*(p+q)) )
        kld_array=np.array([])
        for vector1 in vect1:
            for vector2 in vect2:
                tmp = f( vector1, vector2 )
                kld_array = np.append(kld_array,tmp)
        return kld_array
    
# Janson Shanon Divergence
    def CalcJSD(self,Sarray1,Sarray2,samp):
        dist1 = self._GetPower(Sarray1,samp)
        dist2 = self._GetPower(Sarray2,samp)
        
        self._jsd = self._JSD(dist1,dist2)    
        np.savez(p_path+'jsd/'+Sarray1+'_VS_'+Sarray2,self._jsd)
        
    def ShowJSD(self,Sarray1,Sarray2,samp):
        tmp = np.load(p_path+'jsd/'+Sarray1+'_VS_'+Sarray2+'.npz')['arr_0']
        print Sarray1+'_VS_'+Sarray2+'Jensen-Shannon Divergence'
        print 'shape ='+str(tmp.shape)
        print tmp
        
        
    def ShowFFT(self,name):
        data = self._String2Var(name)
        print data

In [15]:
test = RecordData()
test.ImportCSV('west.csv')
#test.CalcFFT(512)
#test.CalcPower(512)
test.CalcKLD('power_AccX','power_AccX',16)
test.ShowKLD('power_AccX','power_AccX',16)

test.CalcJSD('power_AccX','power_AccX',16)
test.ShowJSD('power_AccX','power_AccX',16)

#print test._power_AccX.shape
#d=test._Query('AccX',[test._Time[0],test._Time[0]+datetime.timedelta(seconds=0.16)])
#fftpack.fft(test._AccX)
#print test._Time[0]
#print type(test._Time[0])
#print test._Time[0]+np.timedelta64(160,'ms')
#np.where(test._Time==np.datetime64('2016-05-20T18:06:15.908000000') )
#min(test._Time)> max(test._Time)
#test.ShowFFT('fft_AccX')
#test.ShowFFT('fft_AccY')
#test.ShowFFT('fft_AccZ')
#test.ShowFFT('fft_GyrX')
#test.ShowFFT('fft_GyrY')
#test.ShowFFT('fft_GyrZ')

NameError: name 'RecordData' is not defined

In [ ]:
data = pd.read_csv("Conv-west-ags.csv",encoding="Shift-JIS")
data.columns=[u'Type',u'Time',u'AccX',u'AccY',u'AccZ',u'GyrX',u'GyrY',u'GyrZ']
data.AccX = data.AccX*0.1*0.001
data.AccY = data.AccY*0.1*0.001
data.AccZ = data.AccZ*0.1*0.001

data.GyrX = data.GyrX*0.1*0.001
data.GyrY = data.GyrY*0.1*0.001
data.GyrZ = data.GyrZ*0.1*0.001

data.Time=pd.to_datetime(data.Time)

data = pd.pivot_table(data,values=[u'AccX',u'AccY',u'AccZ',u'GyrX',u'GyrY',u'GyrZ'],index=[u'Time'])
AccX=data.AccX

In [ ]:
2784/8

In [526]:
x=np.array( [[1,2,3],[4,5,6]])
y=np.array( [[6,5,4],[3,2,1]])

f=lambda x,y: np.sqrt(x**2+y**2)
f(x,y)

array([[ 6.08276253,  5.38516481,  5.        ],
       [ 5.        ,  5.38516481,  6.08276253]])

In [550]:
4.0/15

0.26666666666666666

クラスの問題について、レビューを頂いたのでメモする。
・オブジェクトをセンサデータ１つにしたことに対して問題がある。
センサ間のJS、KLダイバージェンスを調べるときに内部のインスタンスを呼ばないと大変
→１つのクラスですべてのセンサデータを扱える方がいい

問題に対して、クラスを位置から設計するのは大変
・クラスをまとめるクラスを用意する必要がある。

In [28]:
nA,nB=10,20
sA,sB=str(10),str(20)
num = {}
test = {}

In [119]:
num['new']=40

ValueError: need more than 1 value to unpack

In [30]:
len(test)


0

In [102]:
len(num)

1

In [107]:
len(num.items())

1

In [221]:
class AllSensorData:        
    def __init__(self):
            self._DicSensor={}
    
    def regist(self, Sname, Pdata):
        if len(self._DicSensor)==7:
            print "this class has max data"
        else:
            data = SensorData()
            data.ImportCSV(Sname,Pdata)   
            self._DicSensor[Sname]=data
            
    def ExecFFT(self,Sname,samp):
        print self._DicSensor[Sname].ClassName
        self._DicSensor[Sname].CalcFFT(samp)

    def DispFFT(self,Sname,samp):
        print self._DicSensor[Sname].ClassName
        col = self._DicSensor[Sname].GetColumns()
        for axis in col:
            print self._DicSensor[Sname].GetFFT('fft_'+axis,samp)    
            
    def ExecPower(self,Sname,samp):
        print self._DicSensor[Sname].ClassName
        self._DicSensor[Sname].CalcPower(samp)

    def DispPower(self,Sname,samp):
        print self._DicSensor[Sname].ClassName
        col = self._DicSensor[Sname].GetColumns()
        for axis in col:
            print self._DicSensor[Sname].GetPower('power_'+axis,samp)
            
    def ShowData(self,Sname):
        print self._DicSensor[Sname].ShowAllDf()
        
    # Kallback Librar Divergence
    def _KLD(self,vect1,vect2):
        f =lambda p,q : np.sum(p * np.log(p / q), axis=(p.ndim - 1))
        kld_array=np.array([])
        for vector1 in vect1:
            for vector2 in vect2:
                tmp = f( vector1, vector2 )
                kld_array = np.append(kld_array,tmp)
        return kld_array
    
    def ExecKLD(self,Sarray1,Saxis1,Sarray2,Saxis2,samp):
        start = time.time()
        dist1 = self._DicSensor[Sarray1].GetPower('power_'+Saxis1,samp)
        dist2 = self._DicSensor[Sarray1].GetPower('power_'+Saxis2,samp)
        
        kld = self._KLD(dist1,dist2)    
        np.savez(p_path+'kld/'+Sarray1+'_'+Saxis1+'_VS_'+Sarray2+'_'+Saxis2,kld)
        elapsed_time = time.time() - start
        print ("elapsed_time:{0}".format(elapsed_time)) + "[sec]"
        
    def ShowKLD(self,Sarray1,Saxis1,Sarray2,samp):
        tmp = np.load(p_path+'kld/'+Sarray1+'_'+Saxis1+'_VS_'+Sarray2+'_'+Saxis2+'.npz')['arr_0']
        print Sarray1+'_'+Saxis1+'_VS_'+Sarray2+'_'+Saxis2+'_'+'Kullback–Leibler Divergence'
        print 'shape ='+str(tmp.shape)
        print tmp
        
    def _JSD(self,vect1,vect2):
        f =lambda p,q : 0.5*self._KLD(p,(0.5*(p+q)) )+0.5*self._KLD(q,(0.5*(p+q)) )
        kld_array=np.array([])
        for vector1 in vect1:
            for vector2 in vect2:
                tmp = f( vector1, vector2 )
                kld_array = np.append(kld_array,tmp)
        return kld_array
    
# Janson Shanon Divergence
    def ExecJSD(self,Sarray1,Saxis1,Sarray2,samp):
        start = time.time()
        dist1 = self._DicSensor[Sarray1].GetPower('power_'+Saxis1,samp)
        dist2 = self._DicSensor[Sarray1].GetPower('power_'+Saxis2,samp)
        
        jsd = self._JSD(dist1,dist2)    
        np.savez(p_path+'jsd/'+Sarray1+'_'+Saxis1+'_VS_'+Sarray2+'_'+Saxis2,kld)
        elapsed_time = time.time() - start
        print ("elapsed_time:{0}".format(elapsed_time)) + "[sec]"
        
    def ShowJSD(self,Sarray1,Sarray2,samp):
        tmp = np.load(p_path+'jsd/'+Sarray1+'_'+Saxis1+'_VS_'+Sarray2+'_'+Saxis2+'.npz')['arr_0']
        print Sarray1+'_'+Saxis1+'_VS_'+Sarray2+'_'+Saxis2+'_'+'Jensen-Shannon Divergence'
        print 'shape ='+str(tmp.shape)
        print tmp
        

In [222]:
class SensorData:
    
    def __init__(self):
        print "__class__"
        # raw data
        self._RawData={}    
        # fft data
        self._FFTData={}
        # power spectol data
        self._PowerData={}    
        # flag exsist data
        self._Flag_exist_data=False
        
        self._columns=['AccX','AccY','AccZ','GyrX','GyrY','GyrZ']
        self._fft_col=['fft_AccX','fft_AccY','fft_AccZ','fft_GyrX','fft_GyrY','fft_GyrZ']
        self._power_col=['power_AccX','power_AccY','power_AccZ','power_GyrX','power_GyrY','power_GyrZ']
        
    def ImportCSV(self,Sclass,csv_file):
        self.ClassName=Sclass
        self.Flag_exist_data=True
        # design dataframe
        data = pd.read_csv(csv_file,encoding="SHIFT-JIS")
        data.columns=[u'Type',u'Time',u'AccX',u'AccY',u'AccZ',u'GyrX',u'GyrY',u'GyrZ']
        data.Time=pd.to_datetime(data.Time)
        data = pd.pivot_table(data,values=[u'AccX',u'AccY',u'AccZ',u'GyrX',u'GyrY',u'GyrZ'],index=[u'Time'])
        
        # convert numpy.darray
        AccX=data.AccX.values*0.0001
        AccY=data.AccY.values*0.0001
        AccZ=data.AccZ.values*0.0001
        GyrX=data.GyrX.values*0.01
        GyrY=data.GyrY.values*0.01
        GyrZ=data.GyrZ.values*0.01
        Time=data.index.to_pydatetime().astype('datetime64[ns]')
        
        # regist each raw data 
        self._RawData['AccX'] = AccX
        self._RawData['AccY'] = AccY
        self._RawData['AccZ'] = AccZ
        self._RawData['GyrX'] = GyrX
        self._RawData['GyrY'] = GyrY
        self._RawData['GyrZ'] = GyrZ
        self._RawData['Time'] = Time
    
    def ShowFlagExistData(self):
        return self.Flag_exist_data
    
    def GetColumns(self):
        return self._columns

    def ShowAllDf(self):
        print 'AccX : ';print self._RawData['AccX']
        print 'AccY : ';print self._RawData['AccY']
        print 'AccZ : ';print self._RawData['AccZ']
        print 'GyrX : ';print self._RawData['GyrX']
        print 'GyrY : ';print self._RawData['GyrY']
        print 'GyrZ : ';print self._RawData['GyrZ']
        
    def _Time2Num(self,time):
        return np.where(self._RawData['Time']==np.datetime64(time) )[0][0]
    
    def ShowQuery(self,Sname,rng=[]):
        data = self._RawData[Sname]
        print Sname+':'+str( data[rng[0]:rng[1]])
            
    def _sliding_window(self,Sname,samp, overlap):
        count =0
        s =self._RawData['Time'][0]
        start=self._Time2Num(s)
        g = s+np.timedelta64(samp*10,'ms')
        goal= self._Time2Num(g)
        yield self._RawData[Sname][start:goal]
        
        add=overlap*0.01
        
        while True:
            try:
                count +=1
                s =s+np.timedelta64(samp*10,'ms')
                start=self._Time2Num(s)
                g  =s+np.timedelta64(samp*10,'ms')
                goal= self._Time2Num(g)
                yield self._RawData[Sname][start:goal]
            except StopIteration:
                print '_sliding_window StopIteration'
                break   
            except IndexError:
                print '_sliding_window IndexError'
                break  
                
# Fast Frier transaction            
    def GetFFT(self,Sfft,samp):
        return np.load(p_path+'fft/'+self.ClassName+'_'+Sfft+'_'+str(samp)+'.npz')['arr_0']
    
    def CalcFFT(self,samp,overlap=0.5):
        start = time.time()
        fft_data = np.array([])
        
        for n,f in zip( self._columns,self._fft_col):
            print 'start'+n+'->'+f
            sw = self._sliding_window(n,samp,overlap)
            while True:
                try:
                    d=sw.next()
                    fft_data = np.append(fft_data, fftpack.fft(d)[1:(samp/2)+1] )   # fftの直流成分を除くsample/2の
                except StopIteration:
                    print 'CalcFFTStopIteration'
                    fft_data = fft_data.reshape(len(fft_data)/(samp/2),(samp/2) )
                    self._FFTData[f] = fft_data
                    np.savez(p_path+'fft/'+self.ClassName+'_'+str(f)+'_'+str(samp),self._FFTData[f])
                    break   
        elapsed_time = time.time() - start
        print ("elapsed_time:{0}".format(elapsed_time)) + "[sec]"
           

# Spectol Power 
    def GetPower(self,Spower,samp):
        return np.load(p_path+'power/'+self.ClassName+'_'+Spower+'_'+str(samp)+'.npz')['arr_0']
    
    def _power(self,fft_array):
        p=lambda x,y : np.sqrt(x**2+y**2)
        power_array=np.array([])
        for vector in fft_array:
            tmp = p( np.real(vector),np.imag(vector) )
            tmp = tmp/np.sum(tmp)
            power_array = np.append(power_array,tmp)
        return power_array
        
    def CalcPower(self,samp,overlap=0.5):
        start = time.time()
        for fft_name,power_name in zip( self._fft_col, self._power_col):
            print 'start'+fft_name+'->'+power_name
            fft_data = self.GetFFT(fft_name,samp)
            power_data=self._power(fft_data)
            np.savez(p_path+'power/'+self.ClassName+'_'+power_name+'_'+str(samp),power_data)
        elapsed_time = time.time() - start
        print ("elapsed_time:{0}".format(elapsed_time)) + "[sec]"

In [223]:
AllTest = AllSensorData()
AllTest.regist('left_hand','Conv-left-hand-ags.csv')
AllTest.regist('right_leg','Conv-right-leg-ags.csv')

__class__
__class__


In [224]:
AllTest.ShowData('left_hand')

AccX : 
[ 0.8226  0.8058  0.8292 ..., -0.1397 -0.1346 -0.1292]
AccY : 
[ 0.7922  0.8437  0.9091 ...,  0.7067  0.7526  0.8188]
AccZ : 
[-0.0543 -0.0582 -0.0231 ..., -0.5123 -0.4901 -0.5248]
GyrX : 
[ 353.11  361.41  353.95 ...,   -4.59   -2.53   -2.67]
GyrY : 
[ 71.96  59.74  48.03 ...,  56.    57.86  60.15]
GyrZ : 
[ -9.53 -22.7  -34.02 ..., -36.9  -32.73 -29.73]
None


In [225]:
AllTest.ShowData('right_leg')

AccX : 
[ 0.8565  0.8391  0.8926 ...,  0.9554  0.9534  0.9619]
AccY : 
[ 0.0087 -0.0596 -0.0813 ..., -0.28   -0.2688 -0.2673]
AccZ : 
[ 0.209   0.157   0.1709 ...,  0.0203  0.0527  0.0481]
GyrX : 
[-21.33 -23.51 -14.14 ...,   7.81   7.21   6.54]
GyrY : 
[ -4.79   7.47  17.92 ...,   6.63   5.67   5.02]
GyrZ : 
[ 13.29   5.15  -5.34 ...,  -7.26  -7.03  -6.23]
None


In [226]:
AllTest.ExecFFT('left_hand',16)

left_hand
startAccX->fft_AccX
_sliding_window IndexError
CalcFFTStopIteration
startAccY->fft_AccY
_sliding_window IndexError
CalcFFTStopIteration
startAccZ->fft_AccZ
_sliding_window IndexError
CalcFFTStopIteration
startGyrX->fft_GyrX
_sliding_window IndexError
CalcFFTStopIteration
startGyrY->fft_GyrY
_sliding_window IndexError
CalcFFTStopIteration
startGyrZ->fft_GyrZ
_sliding_window IndexError
CalcFFTStopIteration
elapsed_time:5.95280098915[sec]


In [227]:
AllTest.ExecFFT('right_leg',16)

right_leg
startAccX->fft_AccX
_sliding_window IndexError
CalcFFTStopIteration
startAccY->fft_AccY
_sliding_window IndexError
CalcFFTStopIteration
startAccZ->fft_AccZ
_sliding_window IndexError
CalcFFTStopIteration
startGyrX->fft_GyrX
_sliding_window IndexError
CalcFFTStopIteration
startGyrY->fft_GyrY
_sliding_window IndexError
CalcFFTStopIteration
startGyrZ->fft_GyrZ
_sliding_window IndexError
CalcFFTStopIteration
elapsed_time:6.05334711075[sec]


In [228]:
AllTest.DispFFT('left_hand',16)

left_hand
[[ 0.37920144-0.05929202j -0.05606245-0.02899706j  0.04128879-0.0864666j
  ...,  0.01606245+0.02560294j  0.05219851+0.06089044j  0.11930000+0.j        ]
 [ 0.09903043-0.92279313j  0.27451966-0.49979698j  0.15211482-0.28934496j
  ...,  0.17708034-0.09459698j  0.16682011-0.03510812j  0.19270000+0.j        ]
 [ 0.38919818+0.18367701j  0.01575534+0.14228835j -0.00706838+0.11749315j
  ..., -0.05495534+0.03448835j -0.04392128-0.00738628j -0.05050000+0.j        ]
 ..., 
 [-0.41038202-0.19909711j -0.37146435+0.13109855j  0.06954730+0.09766265j
  ..., -0.06613565+0.00989855j -0.03534312-0.02376532j -0.08550000+0.j        ]
 [ 0.25773225+0.66071007j -0.25461936+0.26735258j -0.01388304+0.20446779j
  ..., -0.08958064+0.05155258j -0.09345433+0.02243494j -0.12290000+0.j        ]
 [ 0.30710427+0.04456604j  0.02138894-0.0667804j   0.02723755+0.03917587j
  ..., -0.02018894+0.0826196j  -0.06477277-0.07910414j  0.08690000+0.j        ]]
[[ 0.37920144-0.05929202j -0.05606245-0.02899706j  0.041288

In [229]:
AllTest.DispFFT('right_leg',16)

right_leg
[[ 0.18918139+0.53636481j -0.10848650+0.23996887j -0.18146697+0.18075618j
  ..., -0.08331350+0.08416887j -0.07049860+0.01898412j -0.08560000+0.j        ]
 [-0.29460938-0.40254504j  0.02090725-0.16157155j  0.05883569-0.09217763j
  ...,  0.05329275-0.02977155j  0.06774223-0.00630691j  0.05470000+0.j        ]
 [ 0.87916669+1.30013644j -1.05767381+1.11453425j -1.03616793-0.52221813j
  ...,  0.52907381-0.04906575j -0.05488085+0.2882013j   0.15530000+0.j        ]
 ..., 
 [-0.04969063-0.02721006j  0.02955914+0.12793114j  0.07629629-0.00991074j
  ..., -0.03195914+0.02353114j -0.02192056+0.00235949j -0.02570000+0.j        ]
 [ 0.22240105-0.18728499j  0.04075772-0.10320078j  0.03319202-0.05019173j
  ...,  0.03524228-0.00640078j  0.05850273-0.02156943j  0.05690000+0.j        ]
 [-0.22815089+0.05091498j  0.07373940+0.01295097j -0.01489403+0.03022817j
  ..., -0.00573940+0.00515097j -0.00989675+0.02194959j  0.00670000+0.j        ]]
[[ 0.18918139+0.53636481j -0.10848650+0.23996887j -0.18146

In [ ]:
AllTest.ExecPower('left_hand',16)
AllTest.ExecPower('right_leg',16)

left_hand
startfft_AccX->power_AccX
startfft_AccY->power_AccY
startfft_AccZ->power_AccZ
startfft_GyrX->power_GyrX
startfft_GyrY->power_GyrY
startfft_GyrZ->power_GyrZ
elapsed_time:3.73180413246[sec]
right_leg
startfft_AccX->power_AccX
startfft_AccY->power_AccY
startfft_AccZ->power_AccZ
startfft_GyrX->power_GyrX
startfft_GyrY->power_GyrY
startfft_GyrZ->power_GyrZ
elapsed_time:3.62736821175[sec]


In [ ]:
AllTest.ExecKLD('left_hand','AccX','right_leg','AccX',samp=16)